## 인공지능 작사가 만들기

## 1. 데이터 읽어오기

txt 파일들의 문장들을 줄 단위로 추출하여 'raw_corpus' 리스트에 담는다.(corpus는 '말뭉치'라는 뜻)

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)    # 여러개의 txt 파일을 읽어 리스트에 저장

In [2]:
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:    # 읽기모드("r")로 txt 파일 읽기
        raw = f.read().splitlines()    # txt 파일을 줄 단위로 추출하기
        raw_corpus.extend(raw)    # raw_corpus 리스트에 담기
        
print("데이터 크기:", len(raw_corpus))
print("Examples: \n", raw_corpus[:10])

데이터 크기: 187088
Examples: 
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah Your faith was strong but you needed proof']


## 2. 데이터 전처리: 문장 정제, 토큰화 및 벡터화

띄어쓰기를 기준으로 문장의 단어들을 정의하기에 앞서, 대소문자 통일/문장부호 분리/반복되거나 문장 양 옆에 붙은 공백 삭제/특수문자 삭제/문장의 시작과 끝을 정의하는 '\<start\>'와 '\<end\>' 삽입 등의 정제 과정을 수행할 함수를 정의한다.


각각의 단어들을 텐서플로우가 제공하는 tf.keras.preprocessing.text.Tokenizer 패키지를 통해 숫자로 변환(벡터화(vercorize), 여기서 변환된 숫자를 '텐서(tensor)'라고 칭한다)하고, 각 단어들과 텐서를 매칭한 단어 사전을 정의한다.

In [3]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 모두 소문자로 바꾸기
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백을 넣기
    sentence = re.sub(r'[" "]+', " ", sentence) # 공백들은 하나의 공백으로 바꾸기
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) #a-zA-Z?.!,¿가 아닌 모든 문자는 공백으로 바꾸기 
    sentence = sentence.strip() # 양쪽 공백을 지우기
    sentence = '<start> ' + sentence + ' <end>' # 시작과 끝에 마크 달기
    return sentence

for i in range(10):
    print(preprocess_sentence(raw_corpus[i]))

<start> now i ve heard there was a secret chord <end>
<start> that david played , and it pleased the lord <end>
<start> but you don t really care for music , do you ? <end>
<start> it goes like this <end>
<start> the fourth , the fifth <end>
<start> the minor fall , the major lift <end>
<start> the baffled king composing hallelujah hallelujah <end>
<start> hallelujah <end>
<start> hallelujah <end>
<start> hallelujah your faith was strong but you needed proof <end>


* 문장 전처리 함수 preprocess_sentence(sentence) 생성
  * strip(): 괄호 안에 지정한 문자(열)을 삭제함. 괄호를 비워두면 문장의 앞 뒤에 있는 공백을 삭제함
  * re.sub(찾을 패턴(정규 표현식), 치환할 문자(열), 대상 문자열): 대상 문자열에서 지정된 패턴을 만족하는 부분을 지정된 문자(열)로 치환함


* 정규 표현식이란 특정한 규칙을 가진 문자열의 집합을 표현하는 데 사용하는 일종의 형식 언어이다. 파이썬에서 정규 표현식을 사용할 때는 내장 모듈인 re를 import해야 한다

In [4]:
corpus = []

for sentence in raw_corpus:
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split()) > 15: continue
    corpus.append(preprocessed_sentence)

corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

* raw_corpus에 있는 문장들을 모두 정제하여 입력 데이터 'corpus' 리스트에 담는다.
* 너무 긴 문장은 토큰화 했을 때 비교적 짧은 문장에 지나치게 많은 패딩(padding)을 갖게 하므로, '\<start\>'와 '\<end\>'를 포함하여 단어의 수가 15개를 넘는 문장은 제외하였다.

In [5]:
import tensorflow as tf

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000,
        filters=' ',
        oov_token="<unk>"  
    )    #15000단어까지 저장하는 tokenizer 생성

    tokenizer.fit_on_texts(corpus)    # corpus를 기반으로 tokenizer 내부 단어장 생성
    tensor = tokenizer.texts_to_sequences(corpus)    # 단어들을 숫자로 변환(corpus를 tensor로 변환)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    # 입력 데이터의 시퀀스 길이를 만추어, 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙임
    
    print(tensor, '\n', tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2971 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  117 ...    0    0    0]
 [   2  258  195 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] 


In [6]:
# tokenizer 내부 단어장 인덱스와 단어 확인

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


## 3. 평가 데이터 분할과 데이터셋 객체 생성

모델의 입력이 되는 문장을 Source Sentence, 출력 문장(모델이 맞추어야 할 '정답' 역할)을 Target Sentence라고 한다. 타겟 문장은 첫 단어 '\<start\>'를 입력 받은 다음 생성될 문장에 해당하므로, 모델을 학습시킬 때는 첫번째 토큰을 잘라낸 것을 타겟 문장(dec_train)으로 지정한다. 반대로 소스 문장(enc_train)은 마지막 토큰을 잘라낸 것으로 지정한다.

데이터 분할이 끝난 다음, 모델을 학습시키기 위해 데이터셋 객체를 생성한다. 텐서플로우에서 제공하는 기능을 이용해 텐서로 생성된 데이터들의  tf.data.Dataset 객체를 생성하면 데이터 입력 시 파이프라인을 통한 속도 개선 등의 이점을 누릴 수 있다. 

In [7]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(tensor[:,:-1], tensor[:,1:], test_size = 0.2, random_state = 50)

In [8]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (133863, 14)
Target Train: (133863, 14)


In [9]:
print(enc_train[0])
print(dec_train[0])

[   2 2020  363 2020   73 1155   10 7025    3    0    0    0    0    0]
[2020  363 2020   73 1155   10 7025    3    0    0    0    0    0    0]


In [10]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전에 <pad> 추가

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)

* 텐서 형태로 생성되어 있던 데이터에 tf.data.Dataset.from_tensor_slices() 메소드를 이용해 tf.data.Dataset 객체를 형성한다.
* Batch size란 머신러닝에서 한번 파라미터가 업데이트 될 때 이용되는 학습 데이터의 수를 의미한다. [용어설명 참고](https://radiopaedia.org/articles/batch-size-machine-learning) 일종의 학습 단위의 역할을 하는 수치로 보인다.

## 4. 모델 생성과 학습시키기

모델은 Embedding 레이어->2개의 LSTM 레이어->Dense 레이어로 구성되어 있다.

Embedding 레이어에서는 단어들의 인덱스를 워드 벡터로 변환하여 단어를 추상적으로 표현한다. 2개의 LSTM 레이어를 거치며 모델은 문장을 순차적으로 읽으며 단어 간의 연관성을 분석하고, Dense 레이어에서는 그 결과를 바탕으로 다음에 생성할 단어를 결정한다.

In [11]:
class TextGenerator(tf.keras.Model):    # tf.keras.Model의 서브클래스임
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256   # 단어가 추상적으로 표현되는 크기
hidden_size = 1024    # hidden layer 깊이

lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
# modal 'lyricist'가 제대로 build 될 수 있도록 데이터셋에서 한 배치를 가져와 호출
for src_sample, tat_sample in dataset.take(1): break
    
lyricist(src_sample)

<tf.Tensor: shape=(256, 14, 15001), dtype=float32, numpy=
array([[[ 2.30684891e-05, -7.96226959e-05, -1.82434960e-04, ...,
          8.52816302e-05, -1.12183452e-05,  1.61298231e-05],
        [ 1.79620649e-04, -1.59299234e-04, -3.24988505e-04, ...,
          9.01273233e-05,  1.08611093e-04, -2.51433492e-04],
        [ 2.63709167e-04, -1.32490619e-04, -3.35847784e-04, ...,
          7.67944803e-05,  2.43297909e-04, -4.93796426e-04],
        ...,
        [ 1.11505250e-03,  1.16501143e-03,  5.43273112e-04, ...,
         -1.60668697e-03,  1.01376360e-03, -6.46587359e-05],
        [ 1.23739755e-03,  1.22543436e-03,  5.82762179e-04, ...,
         -1.76089141e-03,  1.08829734e-03,  7.07973595e-05],
        [ 1.35261496e-03,  1.26741396e-03,  6.07508060e-04, ...,
         -1.88889750e-03,  1.16379431e-03,  1.98678448e-04]],

       [[ 2.30684891e-05, -7.96226959e-05, -1.82434960e-04, ...,
          8.52816302e-05, -1.12183452e-05,  1.61298231e-05],
        [-2.40670313e-04, -1.22190235e-04, -4

In [13]:
lyricist.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3840256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  15376025  
Total params: 32,855,961
Trainable params: 32,855,961
Non-trainable params: 0
_________________________________________________________________


In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=10)

Epoch 1/10
522/522 [==============================] - 94s 171ms/step - loss: 3.3279
Epoch 2/10
522/522 [==============================] - 97s 186ms/step - loss: 2.8702
Epoch 3/10
522/522 [==============================] - 97s 186ms/step - loss: 2.7198
Epoch 4/10
522/522 [==============================] - 98s 187ms/step - loss: 2.6065
Epoch 5/10
522/522 [==============================] - 98s 187ms/step - loss: 2.5101
Epoch 6/10
522/522 [==============================] - 98s 186ms/step - loss: 2.4244
Epoch 7/10
522/522 [==============================] - 97s 186ms/step - loss: 2.3450
Epoch 8/10
522/522 [==============================] - 98s 186ms/step - loss: 2.2708
Epoch 9/10
522/522 [==============================] - 97s 186ms/step - loss: 2.2004
Epoch 10/10
522/522 [==============================] - 97s 186ms/step - loss: 2.1339


## 5. 문장 생성하기

In [23]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):

    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [28]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so much <end> '

In [29]:
generate_text(lyricist, tokenizer, init_sentence="<start> he", max_len=20)

'<start> he s a sportsman and a shepherd <end> '

In [26]:
generate_text(lyricist, tokenizer, init_sentence="<start> she", max_len=20)

'<start> she s a sportsman and a shepherd <end> '

In [27]:
generate_text(lyricist, tokenizer, init_sentence="<start> you", max_len=20)

'<start> you re the one that i m in <end> '

In [21]:
generate_text(lyricist, tokenizer, init_sentence="<start> we", max_len=20)

'<start> we re gonna have to be the same <end> '

* init_sentence에 따라 부자연스럽게 종결된 문장도 있고 중복된 문장도 있지만, 주어가 바뀜에 따라 be동사를 바꾸고, 동사 뒤에는 명사나 부사가 오는 등 기본적인 영어 문법에 맞춰 문장을 생성하고 있다.

## 6. 하이퍼파라미터 바꿔보기
 
RNN(순환신경망) 구조에서 입력층과 출력층 사이의 은닉층(hidden layer)은 한 계층을 지날 때 마다 각기 예측한 값을 다음 계층에 넘기며 학습을 진행한다. 그 과정에서 모델은 단어와 단어 사이의 규칙을 도출하게 된다.

embedding_size를 단어를 추상적으로 표현된 정도라고 한다면, 단어의 추상적인 정도가 높아질 수록 규칙을 도출하는 과정이 길어질 것이라고 예상하는 것이 자연스럽다. 이에 hidden_size는 바꾸지 않고, embedding_size를 조절하며 결과를 비교해본다.

In [50]:
embedding_size = 128
hidden_size = 1024

lyricist_2 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [51]:
for src_sample, tat_sample in dataset.take(1): break
    
lyricist_2(src_sample)
lyricist_2.summary()

Model: "text_generator_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  1920128   
_________________________________________________________________
lstm_6 (LSTM)                multiple                  4722688   
_________________________________________________________________
lstm_7 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_3 (Dense)              multiple                  15376025  
Total params: 30,411,545
Trainable params: 30,411,545
Non-trainable params: 0
_________________________________________________________________


In [52]:
lyricist_2.compile(loss=loss, optimizer=optimizer)
lyricist_2.fit(dataset, epochs=10)

Epoch 1/10
522/522 [==============================] - 98s 185ms/step - loss: 3.1847
Epoch 2/10
522/522 [==============================] - 96s 183ms/step - loss: 2.7731
Epoch 3/10
522/522 [==============================] - 96s 184ms/step - loss: 2.6324
Epoch 4/10
522/522 [==============================] - 96s 184ms/step - loss: 2.5167
Epoch 5/10
522/522 [==============================] - 96s 183ms/step - loss: 2.4086
Epoch 6/10
522/522 [==============================] - 96s 183ms/step - loss: 2.3073
Epoch 7/10
522/522 [==============================] - 96s 183ms/step - loss: 2.2113
Epoch 8/10
522/522 [==============================] - 96s 184ms/step - loss: 2.1202
Epoch 9/10
522/522 [==============================] - 96s 183ms/step - loss: 2.0317
Epoch 10/10
522/522 [==============================] - 96s 184ms/step - loss: 1.9465


In [53]:
generate_text(lyricist_2, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [54]:
generate_text(lyricist_2, tokenizer, init_sentence="<start> he", max_len=20)

'<start> he s got a multi bonnet <end> '

In [55]:
generate_text(lyricist_2, tokenizer, init_sentence="<start> she", max_len=20)

'<start> she s got a multi garden <end> '

In [56]:
generate_text(lyricist_2, tokenizer, init_sentence="<start> you", max_len=20)

'<start> you re the only one i want <end> '

In [57]:
generate_text(lyricist_2, tokenizer, init_sentence="<start> we", max_len=20)

'<start> we re gonna have to serve somebody <end> '

* embedding size를 절반으로 줄인 결과 파라미터는 30,411,545개로 다소 줄어들었다.
* validation loss가 약 1.9로 줄어들었다. embedding size를 줄이는 것이 '정답률'을 낮추지는 않는 것으로 보이나, 다소 어색한 단어 조합을 출력하고 있다.('multi bonnet', 'multi garden')
* epoch 당 소요시간은 거의 줄지 않았다. embedding size를 줄여 단어들의 추상적인 정도가 줄어들었다고 해서 학습시간이 크게 줄지는 않는 것으로 보인다.

In [40]:
embedding_size = 512
hidden_size = 1024   
lyricist_3 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [43]:
for src_sample, tat_sample in dataset.take(1): break
    
lyricist_3(src_sample)
lyricist_3.summary()

Model: "text_generator_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  7680512   
_________________________________________________________________
lstm_4 (LSTM)                multiple                  6295552   
_________________________________________________________________
lstm_5 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_2 (Dense)              multiple                  15376025  
Total params: 37,744,793
Trainable params: 37,744,793
Non-trainable params: 0
_________________________________________________________________


In [44]:
lyricist_3.compile(loss=loss, optimizer=optimizer)
lyricist_3.fit(dataset, epochs=10)

Epoch 1/10
522/522 [==============================] - 104s 195ms/step - loss: 3.2980
Epoch 2/10
522/522 [==============================] - 103s 197ms/step - loss: 2.8817
Epoch 3/10
522/522 [==============================] - 103s 197ms/step - loss: 2.7028
Epoch 4/10
522/522 [==============================] - 103s 198ms/step - loss: 2.5672
Epoch 5/10
522/522 [==============================] - 103s 198ms/step - loss: 2.4453
Epoch 6/10
522/522 [==============================] - 103s 197ms/step - loss: 2.3306
Epoch 7/10
522/522 [==============================] - 103s 198ms/step - loss: 2.2196
Epoch 8/10
522/522 [==============================] - 103s 198ms/step - loss: 2.1135
Epoch 9/10
522/522 [==============================] - 103s 197ms/step - loss: 2.0111
Epoch 10/10
522/522 [==============================] - 103s 198ms/step - loss: 1.9133


In [45]:
generate_text(lyricist_3, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

In [46]:
generate_text(lyricist_3, tokenizer, init_sentence="<start> he", max_len=20)

'<start> he s got a hungry heart <end> '

In [47]:
generate_text(lyricist_3, tokenizer, init_sentence="<start> she", max_len=20)

'<start> she s got me runnin round and round <end> '

In [48]:
generate_text(lyricist_3, tokenizer, init_sentence="<start> you", max_len=20)

'<start> you re the only one i want to be <end> '

In [49]:
generate_text(lyricist_3, tokenizer, init_sentence="<start> we", max_len=20)

'<start> we re gonna have to fight <end> '

* embedding size를 두배로 늘린 결과 파라미터는 37,744,793개로 늘었다..
* validation loss가 약 1.9까지 내려갔고, init_sentence별로 중복된 문장 또는 단어가 나타나지 않으며, 맥락상 크게 어색하지 않다.
* epoch 당 소요시간은 100초 대로 늘었다. 단어들의 추상적인 정도가 일정 수준 이상으로 높아지면 학습시간에도 영향이 있는 것으로 보인다.

## 회고

* 알게 된 점
  * 단어는 단어 간의 사이에서 의미와 맥락을 가진다. 이는 자연어 처리에서는 의미 벡터(추상화)와 확률분포로 나타난다.이를 구현한 모델에서는 각 하이퍼파라미터를 무작정 높인다는 것이 효과적이지 않을 수도 있다. 
  

* 궁금한 점 
  * 문장 데이터 전처리 시 단어의 수가 15개를 넘는 문장은 제외했는데, 만약 그보다 더 긴 문장을 허용한다면 모델은 더 긴 문장을 작성할 수 있을까?
  * tokenizer 내 저장할 단어의 수를 더 늘리면 처리해야 할 단어간의 관계, 의미의 폭이 더 넓어지니 validation loss를 낮추기 위해 embedding size나 hidden size를 수정해야 할까? 아니면 차이가 없을까?
  

* 모호한 점
  * 노드 내용에서는 embedding size와 hidden size를 각각 2의 8제곱, 2의 10제곱으로 제시했었는데, 하이퍼파라미터로 2의 거듭제곱 수를 사용하는 것에 이점이 있는지 궁금하다.
  * 자연어 처리를 위한 딥러닝에서 하이퍼파라미터는 문장의 자연스러운 맥락을 결정짓고, 그러므로 적절한 값을 찾는 것이 무척 중요한 과제일 듯 하다. 하지만 문장의 '자연스러운 맥락'을 어떻게 정의하고, 어떤 원칙으로 하이퍼파라미터를 조정해야 할까?


* 다짐: 텐서라는 개념은 분명 학부 전공에서 접해본 적이 있는 개념인데... 잘 사용해본 적이 없어서 기억이 가물가물하다. 단어를 '차원화'하여 사용하는 자연어 처리 공부를 위해서는 꼭 다시 알아두어야 할 것 같다.